In [ ]:
%matplotlib inline

import healpy as hp
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pickle
from itertools import combinations, combinations_with_replacement
from importlib import reload

# Specific qubic modules
from qubicpack.utilities import Qubic_DataDir
import pysm3
import qubic
from qubic import camb_interface as qc
from qubic import NamasterLib as nam
from qubic import mcmc

rc('figure', figsize=(16, 10))
rc('font', size=18)
plt.rcParams['image.cmap'] = 'jet'

In [ ]:
import seaborn as sns
palet = sns.cubehelix_palette(6, start=2, hue=1, light=0.75)
sns.palplot(palet)

In [ ]:
config = 'FI220'
nside = 256 # Define here to avoid loading a diccionary

## Coverage and seenmap

In [ ]:
DataFastSimCoverage = pickle.load(open('../../doc/FastSimulator/Data/DataFastSimulator_' + config +'_coverage.pkl',
                                       "rb"))
coverage = DataFastSimCoverage['coverage']
seenmap = coverage > np.max(coverage) * 0.1

## Theoretical spectra with Camb

In [ ]:
# Make a Namaster object (needed to bin the Camblib)
lmin = 40
lmax = 2 * nside - 1
delta_ell = 30


mask = np.zeros(12 * nside ** 2)
mask[seenmap] = 1
Namaster = nam.Namaster(mask, lmin=lmin, lmax=lmax, delta_ell=delta_ell)
mask_apo = Namaster.get_apodized_mask()
# hp.gnomview(mask_apo, reso=20, title='Mask')

ell_binned, b = Namaster.get_binning(nside)
nbins = len(ell_binned)
print('lmin:', lmin)
print('lmax:', lmax)
print('delta_ell:', delta_ell)
print('nbins:', nbins)
print('ell binned:', ell_binned)
print('Fsky: {}'.format(Namaster.fsky))

In [ ]:
# Make a Camb library
# rvalues = np.arange(0., 0.1, 0.001)
# print(rvalues)
# camblib = qc.rcamblib(rvalues, lmax=lmax, save=global_dir + '/doc/CAMB/camblib_0to1_step001.pkl')

In [ ]:
# Load the CAMB library already saved
with open('../../doc/CAMB/camblib_0to1_step001.pkl', "rb") as file:
    camblib = pickle.load(file)
[lll, rvalues, spec, specunlensed] = camblib
# print(rvalues)

In [ ]:
binned_camblib = qc.bin_camblib(Namaster, '../../doc/CAMB/camblib_0to1_step001.pkl', nside, verbose=True)

[lll_b, rvalues_b, spec_b, specunlensed_b] = binned_camblib

## Likelihood on r

In [ ]:
def myBBth(ell, r):
    clBB = qc.get_Dl_fromlib(ell, r, lib=binned_camblib, unlensed=False, specindex=2)[0]
    clBB = np.array(list(clBB) * ncombi)
    return clBB

In [ ]:
nrun = 1
LLH = np.zeros((nrun, 8, len(rvalues)))
sigma68 = np.zeros((nrun, 8))
sigma68_Fisher = np.zeros((nrun, 8))

factor = 10
for r in range(nrun):
    print(f'\n *************Run {r+1}/{nrun}*****************' )
    for index in range(8):
        nbands = index + 1
        print('\n nbands:', nbands)
        # Possible combinations between bands
        combi = list(combinations_with_replacement(np.arange(nbands), 2))
        ncombi = len(combi)
        print('combi:', combi)
        print('ncombi:', ncombi)

        # Covariance noise matrices
        BBcov = np.load('/home/lmousset/QUBIC/Qubic_work/SpectroImagerie/corr_matrices/pourClaudia/'
                               + f'BBcovariance_bincross_nfrecon{nbands}_samereal_' + config + '_v4_clean.npy')

        print(BBcov.shape)

        # =================== Likelihood ========================
        xvals = list(lll_b) * ncombi
        
        # In order to get errorbars, we vary the data randomly
        fakedata = myBBth(lll_b, r=0.)
#         for i in range(fakedata.shape[0]):
#             fakedata[i] = np.random.normal(loc=fakedata[i], scale=np.sqrt(BBcov[i, i])/factor, size=None)
# #             print(fakedata[i])

        error = BBcov#np.sqrt(np.diag(BBcov))
        print(error.shape)

        logLLH = mcmc.LogLikelihood(xvals=xvals, 
                                    yvals=fakedata, 
                                    errors=error,
                                    nbins=nbins,
                                    model=myBBth, 
                                    flatprior=[[0, 1]], 
                                    covariance_model_funct=Namaster.knox_covariance)

        LLH[r, index, :] , sigma68[r, index] = logLLH.compute_sigma68(logLLH, rvalues)
        

        # ====================== Fisher Analysis =======================
#         sigma68_Fisher[r, index] = logLLH.fisher_analysis()
    print(sigma68[r, :])
        
        

In [ ]:
datadir = os.environ['DATA_SPECTROIM']
# print(datadir)
# np.save(datadir + '/FastSimulator/sigma68pureCMB_samev4clean_SigmaOver10_' + config + '_with6-7.npy', sigma68)
np.save(datadir + '/FastSimulator/sigma68pureCMB_samev4clean_theo_' + config + '_with6-7.npy', sigma68)

In [ ]:
sigma68_FI220 = np.load(datadir + '/FastSimulator/sigma68pureCMB_samev4clean_SigmaOver10_FI220_with6-7.npy')
sigma68_FI150 = np.load(datadir + '/FastSimulator/sigma68pureCMB_samev4clean_SigmaOver10_FI150_with6-7.npy')

sigma68_FI220_theo = np.load(datadir + '/FastSimulator/sigma68pureCMB_samev4clean_theo_FI220_with6-7.npy')[0]
sigma68_FI150_theo = np.load(datadir + '/FastSimulator/sigma68pureCMB_samev4clean_theo_FI150_with6-7.npy')[0]
print(sigma68_FI150_theo.shape)

sigma68_FI220_avg = np.nanmean(sigma68_FI220, axis=0)
sigma68_FI220_std = np.nanstd(sigma68_FI220, axis=0)
sigma68_FI150_avg = np.nanmean(sigma68_FI150, axis=0)
sigma68_FI150_std = np.nanstd(sigma68_FI150, axis=0)

In [ ]:
allbands = np.arange(1, 9)
print(allbands)
plt.figure(figsize=(12, 8))
# plt.title(config +  'BBcov')#$\sqrt{diag}$')
plt.errorbar(np.array(allbands) + 0.01, 
             sigma68_FI150_theo / sigma68_FI150_theo[0],
             sigma68_FI150_std * 10,
             fmt='ro',
             label='150 GHz')
plt.errorbar(allbands, 
             sigma68_FI220_theo / sigma68_FI220_theo[0],
             yerr=sigma68_FI220_std * 10,
             fmt='bo',
             ls="",
             label='220 GHz')




plt.xlabel('Number of sub-bands', fontsize=20)
plt.ylabel(r'$\sigma (r)$ / $\sigma_{1band}(r)$', fontsize=20)
plt.ylim(0.9, 2)
plt.axhline(y=1, c='k', ls='--', label='Optimal')
plt.grid()
plt.legend(loc='upper left', fontsize=20)
plt.tight_layout()
# plt.savefig('/home/lmousset/QUBIC/Qubic_work/SpectroImagerie/paper_plot/'+
#             f'Likelihood_CMBonly_suboptimal_SameCleanv4_start0.9.pdf', 
#             bbox_inches='tight') 

In [ ]:
x = np.linspace(0., 1, 10000)
xlim = 0.2

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(20, 10))
ax0, ax1 = axs

fig.suptitle(config + ' - Clean Same $\sqrt{diag}$')

for i, nbands in enumerate([1, 2, 3, 4, 5, 8]):
    
    c = palet[i]
    ax0.set_title('Likelihood')
#     ax0.plot(rvalues, allLLH[i] / np.max(allLLH[i]), 'o', color=c, label=f'Likelihood {nbands} bands')
    ax0.plot(x, np.exp(-0.5*(x/allsigma68[i])**2), color=c, label='Gaussian $\mu = 0, \sigma = \sigma(r)$')
    ax0.axvline(x=allsigma68[i], linestyle='--', color=c, 
                label=' $\sigma(r)={0:6.4f}$ at 68% C.L.'.format(allsigma68[i]))

    ax0.set_xlim(0, xlim)
    ax0.set_xlabel('r')
    ax0.set_ylabel('Posterior')
    ax0.legend(fontsize=14)
    
    
    ax1.set_title('Fisher')
    ax1.plot(x, np.exp(-0.5*(x/allsigma68_Fisher[i])**2), color=c, label=f'Fisher gaussian {nbands} bands')
    ax1.axvline(x=allsigma68_Fisher[i], linestyle='--', color=c, 
                label=' $\sigma_(r)={0:6.4f}$ at 68% C. L.'.format(allsigma68_Fisher[i]))

    ax1.set_xlim(0, xlim)
    ax1.set_xlabel('r')
    ax1.set_ylabel('Posterior')

    ax1.legend(fontsize=14)

In [ ]:
### Figure Spectroim paper
fig = plt.figure(figsize=(12, 10))
ax = plt.gca()
ax0, ax1 = axs
for i, nbands in enumerate([1, 2, 3, 4, 5, 8]):
    if i == 0:
        bandword = 'band'
    else:
        bandword = 'bands'
    c = palet[i]
    ax.set_title('QUBIC - ' + config[-3:] +' GHz')
    ax.plot(x, np.exp(-0.5*(x/allsigma68[i])**2), color=c, label=f'{nbands} sub-{bandword}')
    ax.axvline(x=allsigma68[i], linestyle='--', color=c, 
                label=' $\sigma(r)={0:6.3f}$ at 68% C.L.'.format(allsigma68[i]))

    ax.set_xlim(0, xlim)
    ax.set_xlabel('r')
    ax.set_ylabel('Posterior')
    ax.legend(fontsize=14)
# plt.savefig('/home/lmousset/QUBIC/Qubic_work/SpectroImagerie/paper_plot/'+
#             f'Likelihood_CMBonly_{config}_2y_4xMix.pdf', 
#             bbox_inches='tight') 

In [ ]:
plt.figure(figsize=(20, 20))
plt.suptitle(f'Mix reals - With sample variance - Config {config}')
# plt.suptitle('Do not mix reals - No Bias model - With sample variance - Config FI-150')
for i, nbands in enumerate([1, 2, 3, 4, 5, 8]):
    plt.subplot(3, 2, i+1)
    plt.title(f'{nbands} bands - {config}')
    plt.plot(x, np.exp(-0.5*(x/allsigma68_Fisher[i])**2), color='r', label=f'Fisher gaussian')
    plt.axvline(x=allsigma68_Fisher[i], linestyle='--', color='r', 
                label=' $\sigma_(r)={0:6.4f}$ at 68% C. L. with Fisher'.format(allsigma68_Fisher[i]))

    plt.plot(rvalues, allLLH[i] / np.max(allLLH[i]), 'o', color='b', label='Likelihood')
    plt.plot(x, np.exp(-0.5*(x/allsigma68[i])**2), color='b', label='Gaussian $\mu = 0, \sigma = \sigma(r)$')
    plt.axvline(x=allsigma68[i], linestyle='--', color='b', 
                label=' $\sigma(r)={0:6.4f}$ at 68% C. L. with likelihood'.format(allsigma68[i]))

    plt.xlim(0, xlim)
    plt.xlabel('r')
    plt.ylabel('Posterior')

    plt.legend(fontsize=12)